# Feature Extraction

This sections will cover how to extract regionprops features from an image using `ngio`, `skimage`. Moreover we will also write the features to a table in the ome-zarr container.

# Step 1: Open the OME-Zarr Container

In [ ]:
import numpy as np
import pandas as pd
from skimage import measure


def extract_features(image: np.ndarray, label: np.ndarray) -> pd.DataFrame:
    """Basic feature extraction using skimage.measure.regionprops_table."""
    label = label.squeeze(-1)  # Remove the channel axis if present
    roi_feat_table = measure.regionprops_table(
        label_image=label,
        intensity_image=image,
        properties=[
            "label",
            "area",
            "mean_intensity",
            "max_intensity",
            "min_intensity",
        ],
    )
    return pd.DataFrame(roi_feat_table)

In [ ]:
from pathlib import Path

from ngio import open_ome_zarr_container
from ngio.utils import download_ome_zarr_dataset

# Download the dataset
download_dir = Path(".").absolute().parent.parent / "data"
hcs_path = download_ome_zarr_dataset("CardiomyocyteTinyMip", download_dir=download_dir)
image_path = hcs_path / "B" / "03" / "0"

# Open the ome-zarr container
ome_zarr = open_ome_zarr_container(image_path)

## Step 2: Setup the inputs

In [ ]:
from ngio.transforms import ZoomTransform

# First we will need the image object and the FOVs table
image = ome_zarr.get_image()

# Get the nuclei label
nuclei = ome_zarr.get_label("nuclei")

# In this example we the image is available at an higher resolution than the nuclei
print(f"Image dimensions: {image.dimensions}, pixel size: {image.pixel_size}")
print(f"Nuclei dimensions: {nuclei.dimensions}, pixel size: {nuclei.pixel_size}")

# We need to setup a transform to resample the nuclei to the image resolution
zoom_transform = ZoomTransform(
    input_image=nuclei,
    target_image=image,
    order="nearest",  # Nearest neighbor interpolation for labels
)

## Step 3: Use the FeatureExtractorIterator to create a feature table

In [ ]:
from ngio.experimental.iterators import FeatureExtractorIterator
from ngio.tables import FeatureTable

iterator = FeatureExtractorIterator(
    input_image=image,
    input_label=nuclei,
    label_transforms=[zoom_transform],
    axes_order=["y", "x", "c"],
)

feat_table = []
for image_data, label_data, roi in iterator.iter_as_numpy():
    print(f"Processing ROI: {roi}")
    roi_feat_table = extract_features(image=image_data, label=label_data)
    feat_table.append(roi_feat_table)

# Concatenate all the dataframes into a single one
feat_table = pd.concat(feat_table)
feat_table = FeatureTable(table_data=feat_table, reference_label="nuclei")
ome_zarr.add_table("nuclei_regionprops", feat_table)

### Sanity Check: Read the Table back

In [ ]:
ome_zarr.get_table("nuclei_regionprops").lazy_frame.collect()